In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yarra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data pre-Processing

In [5]:
news_dataset=pd.read_csv('train.csv')

In [6]:
news_dataset.shape

(20800, 5)

In [7]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#.1-fake
#2-true
#counting the number of missing values in dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
#replacing null values with empty string-imputation
news_dataset=news_dataset.fillna('')


In [10]:
#merging author name and title
news_dataset['content']=news_dataset['author']+" "+news_dataset['title']


In [20]:
print(news_dataset['content'])

0        darrel lucu hous dem aide: we didn’t even see ...
1        daniel j. flynn flynn: hillari clinton, big wo...
2          consortiumnews.com whi truth might get you fire
3        jessica purkiss 15 civilian kill in singl us a...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ is sa...
20798    alex ansari nato, russia to hold parallel exer...
20799                    david swanson what keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [12]:
#seperating the data i.e content and label ie 0,1
#axis=1 for removing column,0 for row
X=news_dataset.drop('label',axis=1)
Y=news_dataset['label']

In [13]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

stemming


process of reducing a word to root word

actor,actoress,acting-->act

In [16]:
port_stem=PorterStemmer()

In [22]:
#sub-substitutes certain values
#^-exclusion
def stemming(content):
    #numbers and punctuations replaced by space in the content column
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [23]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [24]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com whi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [25]:
#seperating the data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values
print(X)
print(Y)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [26]:
#converting textual data to numericasl data
#tf-term frequency
#idf-inverse document frequency
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [28]:
print(X)

  (0, 15756)	0.28444970815055715
  (0, 13529)	0.2574970172594486
  (0, 8948)	0.36308461846253576
  (0, 8670)	0.29171397452228925
  (0, 7730)	0.24750334306177618
  (0, 7037)	0.22332642021568422
  (0, 5000)	0.2329885731088067
  (0, 3823)	0.2701524723411949
  (0, 3625)	0.3593873678738682
  (0, 2976)	0.24649757831832603
  (0, 2498)	0.3671344982665244
  (0, 271)	0.2697210820716651
  (1, 16876)	0.30235679096829876
  (1, 6848)	0.1915043283616483
  (1, 5536)	0.7182240415512174
  (1, 3592)	0.2651754305615269
  (1, 2829)	0.1919866643107087
  (1, 2237)	0.37034134781375483
  (1, 1904)	0.15606590885460245
  (1, 1506)	0.29559181189298234
  (2, 16700)	0.32977918089995606
  (2, 15681)	0.39168670004387085
  (2, 9661)	0.46528681634250674
  (2, 6002)	0.32758721252421236
  (2, 5422)	0.367773917694985
  :	:
  (20797, 13176)	0.2484029415900853
  (20797, 12398)	0.272799645228117
  (20797, 12194)	0.24793259901425674
  (20797, 10352)	0.07768525275112491
  (20797, 9629)	0.17465916493414385
  (20797, 9560)	0.293

Splitting dataset into training and test data

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the Logistic Regression model

In [32]:
model=LogisticRegression()

In [33]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [34]:
#accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction, Y_train)

In [35]:
print("Accuracy score of the training data:",training_data_accuracy)

Accuracy score of the training data: 0.987139423076923


In [36]:
#accuracy score on the testing data
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction, Y_test)

In [37]:
print(testing_data_accuracy)

0.9786057692307693


Making a Predictive System

In [45]:
X_new=X_test[0]
prediction=model.predict(X_new)
print(prediction)
if prediction[0]==0:
    print("The news is Real")
else:
    print("The news is Fake")

[1]
The news is Fake


In [46]:
print(Y_test[0])

1
